# 6.4. 多输入多输出通道
当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有$3\times h\times w$的形状。我们将这个大小为$3$的轴称为**通道**（channel）维度。

## 6.4.1. 多输入通道
![](../img/6_4.png)


In [6]:
import torch
from d2l import torch as d2l
"""
实现一下多输入通道互相关运算。

简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加。

zip():用于将可迭代的对象作为参数，将对象中对应的元素打包成一个元组，然后返回由这些元组组成的列表
"""


def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x,k) for x, k in zip(X,K))

# 构造与上图中的值相对应的输入张量X和核张量K，以验证互相关运算的输出。
X=torch.tensor([
                [[0.0,1.0,2.0],
                 [3.0,4.0,5.0],
                 [6.0,7.0,8.0]],

                [[1.0,2.0,3.0],
                 [4.0,5.0,6.0],
                 [7.0,8.0,9.0]]

                ])


K=torch.tensor([
                [[0.0,1.0],
                 [2.0,3.0]],

                [[1.0,2.0],
                 [3.0,4.0]]

                ])

corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 6.4.2. 多输出通道
在最流行的神经网络架构中，随着神经网络层数的加深，我们常会**增加输出通道的维数**，通过减少空间分辨率以获得更大的通道深度。直观地说，我们可以**将每个通道看作是对不同特征的响应**。

而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

用$c_i$和$c_o$分别表示输入和输出通道的数目，并让$k_h$和$k_w$为卷积核的高度和宽度。为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为$c_i\times k_h\times k_w$的卷积核张量，这样卷积核的形状是$c_o\times c_i\times k_h\times k_w$。在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。

In [7]:
"""
实现一个计算多个通道的输出的互相关函数。
"""
def corr2d_multi_in_out(X,K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X,k) for k in K], 0)

# 通过将核张量K与K+1（K中每个元素加）和K+2连接起来，构造了一个具有3个输出通道的卷积核。
K=torch.stack((K,K+1,K+2),0)
K.shape  # torch.Size([3, 2, 2, 2])

# 下面，我们对输入张量X与卷积核张量K执行互相关运算。现在的输出包含3个通道，第一个通道的结果与先前输入张量X和多输入单输出通道的结果一致。
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 6.4.3.  1x1卷积层
$1\times 1$卷积的唯一计算发生在通道上（可缩小通道数）。下图展示了使用$1\times 1$卷积核与$3$个输入通道和$2$个输出通道的互相关计算。这里输入和输出具有相同的高度和宽度，输出中的每个元素都是从输入图像中同一位置的元素的线性组合。我们可以将$1\times 1$卷积层看作是在每个像素位置应用的全连接层，以$c_i$个输入值转换为$c_o$个输出值。因为这仍然是一个卷积层，所以跨像素的权重是一致的。同时，$1\times 1$卷积层需要的权重维度为$c_o\times c_i$，再额外加上一个偏置。

![](../img/6_5.png)

In [9]:
"""
下面，我们使用全连接层实现1x1卷积。

请注意，我们需要对输入和输出的数据形状进行调整。
"""
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]
    X=X.reshape((c_i,h*w))
    K=K.reshape((c_o,c_i))
    # 全连接层中的矩阵乘法
    Y=torch.matmul(K,X)
    return Y.reshape((c_o,h,w))

"""
当执行1x1卷积运算时，上述函数相当于先前实现的互相关函数corr2d_multi_in_out。

让我们用一些样本数据来验证这一点。
"""
X=torch.normal(0,1,(3,3,3))
K=torch.normal(0,1,(2,3,1,1))

Y1=corr2d_multi_in_out_1x1(X,K)
Y2=corr2d_multi_in_out(X,K)
# Python assert（断言）用于判断一个表达式，在表达式条件为 false 的时候触发异常。
assert float(torch.abs(Y1-Y2).sum()) < 1e-6

![](../img/6_12.png)

## 6.4.4. 小结
- 多输入多输出通道可以用来扩展卷积层的模型。
- 每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果。
- 每个输出通道有独立的三维卷积核。
- 当以每像素为基础应用时，**1x1卷积层相当于全连接层**。
- 1x1卷积层通常用于调整网络层的通道数量和控制模型复杂性。